In [24]:
# !/bin/python3
# https://adventofcode.com/2022/day/22

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [25]:
import math
import re
import sys

from itertools import zip_longest
from utils import read_input

In [26]:
def parse(input):
    numbers_pattern = re.compile("(\d+)")
    turns_pattern = re.compile("([RL])")
    board_by_row = {}
    board_by_col = {}

    in_board = True
    for i, line in enumerate(input, start=1):
        if line == "":
            in_board = False
            continue

        if in_board:
            board_by_row[i] = {}
            for j, value in enumerate(line, start=1):
                if value == " ":
                    continue

                if j not in board_by_col:
                    board_by_col[j] = {}

                if "start" not in board_by_row[i]:
                    board_by_row[i]["start"] = j

                if "start" not in board_by_col[j]:
                    board_by_col[j]["start"] = i

                board_by_row[i]["end"] = j
                board_by_col[j]["end"] = i

                board_by_row[i][j] = value
                board_by_col[j][i] = value
            continue

        numbers = [int(value) for value in numbers_pattern.findall(line)]
        turns = turns_pattern.findall(line)
        directions = tuple(zip_longest(numbers, turns))

    return board_by_row, board_by_col, directions

In [27]:
def reverse(val, size):
    return size - val + 1

def get_adjustment(val, middle, size, distance):
    # close = 0, medium = 1, far = 2
    if val < middle:
        return middle - val - (size * distance)
    return (size * (distance + 2)) - val + middle

def get_close_static(val, middle, size):
    if val < middle:
        return middle - 1
    return middle + size + 1
    
    
def get_far_static(val, middle, size):
    if val > middle:
        return middle - 1
    return middle + size + 1
    
adjustments = {
    (0, 0): lambda row, col, middle, size: (row, col - middle + 1, 1),
    (0, 1): lambda row, col, middle, size: (row, get_close_static(col, middle, size), get_close_adjustment(col, middle)),
    (0, 2): lambda row, col, middle, size: (row, get_middle_adjustment(col, middle, size), size + 1),
    (0, 3): lambda row, col, middle, size: (row, get_far_static(col, middle, size), get_far_adjustment(col, middle, size)),

    (1, 0): lambda row, col, middle, size: (size, get_middle_adjustment(col, middle), row - size + 1),
    (1, 1): lambda row, col, middle, size: (get_close_adjustment(col, middle)),
    (1, 2): lambda row, col, middle, size: (),
    (1, 3): lambda row, col, middle, size: (),

    (2, 0): lambda row, col, middle, size: (),
    (2, 1): lambda row, col, middle, size: (),

    (3, 0): lambda row, col, middle, size: (),
    (3, 1): lambda row, col, middle, size: (),

    (1, 0): lambda row, col, middle, size: (size + 1, get_middle_adjustment(col, middle), get_close_adjustment(row, 1)),
    (3, 0): lambda row, col, middle, size: (0, row - (size * 3)),
}

def get_faces(board_by_row, board_by_col):
    size = max(len(board_by_row.keys()), len(board_by_col.keys())) // 4
    middle_col = max(board_by_col.items(), key=lambda data: len(data[1]))
    col = middle_col
    adjustments = [0, 0, 0]

    for row, cols  in board_by_row.items():
        row_val = math.ceil((row - 1) / size)
        if row_val == 0:
            adjustments[2] += 0
        elif row_val == 1:
            adjustments[2] += 1
        elif row_val == 2:
            adjustments[2] = size + 1
        elif row_val == 3:
            adjustments[2] -= 1
            
        for col, value in cols.items():
            if type(col) == str:
                continue

            col_val = abs(math.ceil((col - middle_col) / size))
            if col_val == 1:
                if row_val == 0:
                    adjustments[2] += 1
            elif col_val == 2:
                pass
            elif col_val == 3:
                pass


def get_faces_old(board_by_row, board_by_col):
    size = max(len(board_by_row.keys()), len(board_by_col.keys())) // 4
    faces = [
        {"top": 1, "bottom": size, "left": (size * 2) + 1, "right": size * 3},
        {"top": size + 1, "bottom": size * 2, "left": 1, "right": size + 3},
        {"top": size + 1, "bottom": size * 2, "left": size + 1, "right": size * 2},
        {
            "top": size + 1,
            "bottom": size * 2,
            "left": (size * 2) + 1,
            "right": size * 3,
        },
        {
            "top": (size * 2) + 1,
            "bottom": (size * 3),
            "left": (size * 2) + 1,
            "right": size * 3,
        },
        {
            "top": (size * 2) + 1,
            "bottom": (size * 3),
            "left": (size * 3) + 1,
            "right": size * 4,
        },
    ]

    config = {}

    for row in range(1, len(board_by_row.keys()), size):
        for col in range(board_by_row[row]["start"], board_by_row[row]["end"], size):
            config = update_config(board_by_row, config, row, col, size, (-1, 0))
            config = update_config(
                board_by_col, config, col, row, size, (0, -1), a_is_row=False
            )

    import pprint

    pprint.pprint(config)

    return faces, config


def update_config(board, config, a, b, size, diff, a_is_row=True):
    coord = (a, b) if a_is_row else (b, a)
    if config.get(coord, {}).get(diff):
        return config

    curr_b = None
    if board.get(a - size):
        curr_a = a - size
    elif board.get(a + (size * 3)):
        curr_a = a + (size * 3)
    else:
        curr_a = a + size
        coord_a_data = board[curr_a]
        if coord_a_data.get(b - (size * 2)):
            curr_b = b - (size * 2)
        else:
            curr_b = b + (size * 2)

    if not curr_b:
        min_b = board[curr_a]["start"]
        max_b = board[curr_a]["end"] - size + 1

        if max_b <= b:
            curr_b = max_b
        else:
            curr_b = min_b

    curr_coord = (curr_a, curr_b) if a_is_row else (curr_b, curr_a)
    return add_face(config, coord, curr_coord, *diff)


def add_face(config, coords_a, coords_b, row_offset, col_offset):
    if coords_a not in config:
        config[coords_a] = {}

    if coords_b not in config:
        config[coords_b] = {}

    config[coords_a][(row_offset, col_offset)] = coords_b
    config[coords_b][(-row_offset, -col_offset)] = coords_a

    return config


def move(
    board_by_row, board_by_col, row, col, steps, row_steps, col_steps, wrap, faces=None
):
    for _ in range(steps):
        curr_row = row + row_steps
        curr_col = col + col_steps

        curr_row, curr_col = wrap(
            board_by_row,
            board_by_col,
            curr_row,
            curr_col,
            row_steps,
            col_steps,
            faces=faces,
        )
        if board_by_row.get(curr_row, {}).get(curr_col) != ".":
            break

        row = curr_row
        col = curr_col

    return row, col


def wrap(board_by_row, board_by_col, row, col, row_steps, col_steps, faces=None):
    pass


def move(
    board_by_row, board_by_col, row, col, steps, row_steps, col_steps, wrap, faces=None
):
    for _ in range(steps):
        curr_row = row + row_steps
        curr_col = col + col_steps

        curr_row, curr_col = wrap(
            board_by_row,
            board_by_col,
            curr_row,
            curr_col,
            row_steps,
            col_steps,
            faces=faces,
        )
        if board_by_row.get(curr_row, {}).get(curr_col) != ".":
            break

        row = curr_row
        col = curr_col

    return row, col


def wrap(board_by_row, board_by_col, row, col, row_steps, col_steps, faces=None):
    if row_steps != 0:
        if row > board_by_col[col]["end"]:
            row = board_by_col[col]["start"]
        if row < board_by_col[col]["start"]:
            row = board_by_col[col]["end"]

    if col_steps != 0:
        if col > board_by_row[row]["end"]:
            col = board_by_row[row]["start"]
        if col < board_by_row[row]["start"]:
            col = board_by_row[row]["end"]

    return row, col


def turn(current, target):
    if not target:
        return current
    if target == "R":
        return (current + 1) % 4
    return (current - 1) % 4


def wrap_cube(board_by_row, board_by_col, row, col, row_steps, col_steps, faces=None):
    if row_steps != 0:
        if row > board_by_col[col]["end"]:
            row = board_by_col[col]["start"]
        if row < board_by_col[col]["start"]:
            row = board_by_col[col]["end"]

    if col_steps != 0:
        if col > board_by_row[row]["end"]:
            col = board_by_row[row]["start"]
        if col < board_by_row[row]["start"]:
            col = board_by_row[row]["end"]

    return row, col


def turn_cube(current, target):
    if not target:
        return current
    if target == "R":
        return (current + 1) % 4
    return (current - 1) % 4

In [28]:
def traverse(board_by_row, board_by_col, directions):
    row_a = 1
    col_a = board_by_row[row_a]["start"]
    row_b = 1
    col_b = board_by_row[row_b]["start"]
    faces = get_faces(board_by_row, board_by_col)

    turns = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    direction_a = 0
    direction_b = 0

    for steps, turn_direction in directions:
        row_a, col_a = move(
            board_by_row, board_by_col, row_a, col_a, steps, *turns[direction_a], wrap
        )
        direction_a = turn(direction_a, turn_direction)
        row_b, col_b = move(
            board_by_row,
            board_by_col,
            row_a,
            col_a,
            steps,
            *turns[direction_b],
            wrap_cube,
            faces=faces
        )
        direction_b = turn_cube(direction_b, turn_direction)

    part_a = (1000 * row_a) + (4 * col_a) + direction_a
    part_b = (1000 * row_b) + (4 * col_b) + direction_b
    return part_a, part_b

In [29]:
board_by_row, board_by_col, directions = parse(
    read_input(
        parent=__vsc_ipynb_file__,
        ignore_empty=False,
        sample="a",
    )
)
a, b = traverse(board_by_row, board_by_col, directions)

print("part a:", a)
print("part b:", b)

(9, {'start': 1, 'end': 12, 1: '.', 2: '.', 3: '#', 4: '.', 5: '.', 6: '#', 7: '.', 8: '.', 9: '.', 10: '.', 11: '.', 12: '.'})
part a: 6032
part b: 6032


In [30]:
board_by_row, board_by_col, directions = parse(
    read_input(parent=__vsc_ipynb_file__, ignore_empty=False)
)
a, b = traverse(board_by_row, board_by_col, directions)

print("part a:", a)
print("part b:", b)

(51, {'start': 1, 'end': 150, 1: '.', 2: '.', 3: '.', 4: '.', 5: '.', 6: '.', 7: '.', 8: '.', 9: '.', 10: '.', 11: '.', 12: '.', 13: '.', 14: '#', 15: '.', 16: '#', 17: '.', 18: '#', 19: '.', 20: '.', 21: '#', 22: '.', 23: '.', 24: '.', 25: '.', 26: '.', 27: '.', 28: '.', 29: '.', 30: '.', 31: '.', 32: '.', 33: '.', 34: '.', 35: '.', 36: '.', 37: '.', 38: '#', 39: '.', 40: '.', 41: '.', 42: '.', 43: '.', 44: '.', 45: '.', 46: '.', 47: '.', 48: '.', 49: '.', 50: '.', 51: '.', 52: '.', 53: '.', 54: '.', 55: '.', 56: '.', 57: '.', 58: '.', 59: '.', 60: '.', 61: '.', 62: '#', 63: '.', 64: '.', 65: '.', 66: '.', 67: '.', 68: '.', 69: '.', 70: '.', 71: '.', 72: '.', 73: '.', 74: '.', 75: '.', 76: '.', 77: '.', 78: '.', 79: '.', 80: '.', 81: '.', 82: '#', 83: '.', 84: '.', 85: '.', 86: '.', 87: '#', 88: '.', 89: '.', 90: '.', 91: '.', 92: '.', 93: '.', 94: '.', 95: '#', 96: '.', 97: '.', 98: '.', 99: '#', 100: '.', 101: '.', 102: '.', 103: '.', 104: '.', 105: '.', 106: '.', 107: '.', 108: '.'